# All pairwise associations between Gene Expression and MicroRNA data in TCGA
Check out more notebooks at our ['Regulome Explorer Repository'](https://github.com/isb-cgc/Community-Notebooks/tree/master/RegulomeExplorer)!


```
Title:   All pairwise associations between Gene Expression and MicroRNA data in TCGA
Author:  Boris Aguilar
Created: 03-11-2020
Purpose: Demonstrate how to use the Regulome explorer functionality to find significant associations from Gene expression and microRNA BigQuery tables
URL:     https://github.com/isb-cgc/Community-Notebooks/blob/master/Correlation-GeneExpression-MicroRNA


```
***

In this notebook, we use BigQuery to find significant associations between all possible pairs of genes and unique identifiers available in the RNAseq and microRNA data available in TCGA. The cohort for this analysis consists of BrCA patients with less than 50 years at the time of diagnosis.

To find significant associations between RNAseq and microRNA, we used Bigquery to compute Pearson correlation between all possible pairs. The necessary Queries are generated using functions implemented in the [Regulome Explorer module](https://github.com/isb-cgc/Community-Notebooks/tree/master/RegulomeExplorer/re_module). 

After nonsignificant are filtered out, we use python functions to compute the exact p-values for further statistical analysis.

### Authentication
The first step is to authorize access to BigQuery and the Google Cloud. For more information see ['Quick Start Guide to ISB-CGC'](https://isb-cancer-genomics-cloud.readthedocs.io/en/latest/sections/HowToGetStartedonISB-CGC.html) and alternative authentication methods can be found [here](https://googleapis.github.io/google-cloud-python/latest/core/auth.html).

In [1]:
#%load_ext autoreload
#%autoreload 2
%matplotlib inline
from google.cloud import bigquery
import seaborn as sns
from scipy import stats
import re_module.bq_functions as regulome
import numpy as np

### Parameters
In this experiment we will use Gene expression data and micro RNA data available in ISB-CGC. The information about the Bigquery tables and their columns are stored in the Features dictionary. The desired significance level and the minimum number of patients for the correlation computations are free parameteres that can be changed.

If you want to test this notebook checking all possible pairs please replace "labels1 = ..." with "labels1 = ' IS NOT NULL '" but know the computations will take several minutes to complete, we recommend to generate the query and copy it in the [console](https://console.cloud.google.com/bigquery).

In [2]:
# information from Bigquery tables
Features = {     'Gene Expression' : { 'table'  : 'isb-cgc.TCGA_hg38_data_v0.RNAseq_Gene_Expression',
                                       'symbol' : 'gene_name',
                                       'study'  : 'project_short_name',
                                       'data'   : 'AVG( LOG10( HTSeq__Counts + 1 ) ) ',
                                       'rnkdata': 'data',
                                       'avgdat' : 'avgdata',  
                                   'patientcode': 'case_barcode',
                                    'samplecode': 'sample_barcode',
                                       'where'  : 'AND HTSeq__Counts IS NOT NULL',
                                       'dattype': 'numeric' },
             'MicroRNA Expression': {  'table'  : 'isb-cgc.TCGA_hg38_data_v0.miRNAseq_Expression',
                                       'symbol' : 'mirna_id',
                                        'study' : 'project_short_name',
                                       'data'   : 'AVG( reads_per_million_miRNA_mapped )',
                                       'rnkdata': 'data',
                                       'avgdat' : 'avgdata',
                                   'patientcode': 'case_barcode',
                                    'samplecode': 'sample_barcode',
                                       'where'  : 'AND reads_per_million_miRNA_mapped IS NOT NULL',
                                       'dattype': 'numeric'}
                
               }

# parameteres for Gene expression data
feat1 = Features['Gene Expression']
cohort1 = feat1['patientcode'] + " IN ( SELECT case_barcode FROM cohort ) "  #" IN UNNEST(@PATIENTLIST) "
labels1 = ' =  \'IGLVV-66\' '  # ' IS NOT NULL ' use this for all pair-wise comaprison. Change IGLVV-66 for your favorite gene 

# parameteres for Gene expression data
feat2 = Features['MicroRNA Expression'] 
cohort2 = feat2['patientcode'] + " IN ( SELECT case_barcode FROM cohort ) " # " IN UNNEST(@PATIENTLIST) "
labels2 = ' IS NOT NULL '

# minimum correlation
significance_level = 0.05 # only 0.05, 0.01, 0.005, and 0.001 are alowed !!

# minimun number of samples
nsamples = 25

### Read cohort
The cohort for this analysis consists of BrCA patients with less than 50 years at the time of diagnosis, with II, IIA, IIB pathological stage, race white or black, and excluding patients with hispanic or latino ethnicity.

In [3]:
cohort = """
cohort AS(
SELECT case_barcode FROM `isb-cgc.TCGA_bioclin_v0.Clinical`
WHERE project_short_name = "TCGA-BRCA" AND age_at_diagnosis <=50
)
"""

### Queries to read tables
The queries to retreive the necessary data, query_t1 for Gene expression and query_t2 for micro RNA, are generated using a function available in the regulome explorer module available in here https://github.com/isb-cgc/Community-Notebooks/tree/master/RegulomeExplorer/re_module.  

In [4]:
query_t1 = regulome.generic_numeric_bqtable ( 'table1' , feat1, cohort1, labels1 )
query_t2 = regulome.generic_numeric_bqtable ( 'table2' , feat2, cohort2, labels2 ) 

### Queries to join tables and perform statistics
The two tables are integrated into one table to compute correlations. The query to perform these operations is obtained by using a function implemented in the regulome explorer module.  'sql' contains the complete query to perform the desired correlations.  You can copy the query (which is printed ) to the Bigquery - Google cloud console to run the query and obtain the results. The computations can take 15 minutes. Alternatively you can go to "Run the Query" section of this notebooks. 

Note: `isb-cgc.functions.significance_level_ttest2` is a [persistent function](https://cloud.google.com/bigquery/docs/reference/standard-sql/user-defined-functions)  that estimates upper bounds of p-vaues ( significance levels ), so we can filter out correlations that are not statistically significant ( alpha > significance_level ). This function only work for N < 500. For N>500 we recomend to use an normal approximation of the t distribution.

In [5]:
summ_query = regulome.get_summarized_table('Gene Expression',feat1,'MicroRNA Expression',feat2)

sql = ( 'WITH' + cohort + ',' + query_t1 + ',' + query_t2 + ',' + summ_query + """
SELECT *,
   `isb-cgc.functions.significance_level_ttest2`(n-2, ABS(correlation)*SQRT( (n-2)/((1+correlation)*(1-correlation)))) as alpha
FROM summ_table
WHERE n > {0} AND ABS(correlation) < 0.9999
GROUP BY 1,2,3,4,5
HAVING alpha <= {1}
ORDER BY alpha ASC, correlation DESC
""".format( str(nsamples) , str(significance_level) ) )

print( sql )  # THIS Query perform the computations

WITH
cohort AS(
SELECT case_barcode FROM `isb-cgc.TCGA_bioclin_v0.Clinical`
WHERE project_short_name = "TCGA-BRCA" AND age_at_diagnosis <=50
)
,
table1 AS (
SELECT
   symbol,
   data AS rnkdata,
   ParticipantBarcode
FROM (
   SELECT
      gene_name AS symbol, 
      AVG( LOG10( HTSeq__Counts + 1 ) )  AS data,
      case_barcode AS ParticipantBarcode
   FROM `isb-cgc.TCGA_hg38_data_v0.RNAseq_Gene_Expression`
   WHERE case_barcode IN ( SELECT case_barcode FROM cohort )     # cohort 
         AND gene_name  =  'IGLVV-66'   # labels 
         AND HTSeq__Counts IS NOT NULL  
   GROUP BY
      ParticipantBarcode, symbol
   )
)
,
table2 AS (
SELECT
   symbol,
   data AS rnkdata,
   ParticipantBarcode
FROM (
   SELECT
      mirna_id AS symbol, 
      AVG( reads_per_million_miRNA_mapped ) AS data,
      case_barcode AS ParticipantBarcode
   FROM `isb-cgc.TCGA_hg38_data_v0.miRNAseq_Expression`
   WHERE case_barcode IN ( SELECT case_barcode FROM cohort )     # cohort 
         AND mirna_id  IS N

### Run the Query
The following steps run BigQuery and generate a table with significant associations.
Note: alpha is an upper bound of the significant level. 

In [7]:
bqclient = bigquery.Client()
df_results = regulome.runQuery ( bqclient, sql, [], [], [], dryRun=False )
df_results


 in runQuery ... 
    the results for this query were previously cached 


,symbol1,symbol2,n,correlation,alpha
0,IGLVV-66,hsa-mir-124-2,323,0.419730,0.001
1,IGLVV-66,hsa-mir-124-3,323,0.403092,0.001
2,IGLVV-66,hsa-mir-5591,323,0.381459,0.001
3,IGLVV-66,hsa-mir-124-1,323,0.329851,0.001
4,IGLVV-66,hsa-mir-3935,323,0.300824,0.005
5,IGLVV-66,hsa-mir-6889,323,0.263086,0.010
6,IGLVV-66,hsa-mir-153-1,323,0.258264,0.010
7,IGLVV-66,hsa-mir-153-2,323,0.239804,0.050
8,IGLVV-66,hsa-mir-1224,323,0.231927,0.050
9,IGLVV-66,hsa-mir-875,323,0.214430,0.050


Now that the non significant correlations ( alpha < user defined significant level ) were filtered out, we can use python to compute p-values.

In [8]:
#  t statistics 
f = lambda n, correlation  : correlation * np.sqrt( (n- 2.0) / (1.0 - correlation**2 ))  
df_results['tscore'] = df_results.apply(lambda x: f(x.n,x.correlation), axis=1)

# computing p values from the two tailed t test
f = lambda n, tscore  : (1.0 - stats.t.cdf(abs(tscore), n-2)) * 2.0  
df_results['p-value'] = df_results.apply(lambda x: f(x.n,x.tscore), axis=1)
    
df_results

,symbol1,symbol2,n,correlation,alpha,tscore,p-value
0,IGLVV-66,hsa-mir-124-2,323,0.419730,0.001,8.285239,3.108624e-15
1,IGLVV-66,hsa-mir-124-3,323,0.403092,0.001,7.891500,4.751755e-14
2,IGLVV-66,hsa-mir-5591,323,0.381459,0.001,7.393456,1.253664e-12
3,IGLVV-66,hsa-mir-124-1,323,0.329851,0.001,6.260119,1.231803e-09
4,IGLVV-66,hsa-mir-3935,323,0.300824,0.005,5.651484,3.512751e-08
5,IGLVV-66,hsa-mir-6889,323,0.263086,0.010,4.885682,1.629772e-06
6,IGLVV-66,hsa-mir-153-1,323,0.258264,0.010,4.789669,2.556001e-06
7,IGLVV-66,hsa-mir-153-2,323,0.239804,0.050,4.425579,1.319548e-05
8,IGLVV-66,hsa-mir-1224,323,0.231927,0.050,4.271795,2.557999e-05
9,IGLVV-66,hsa-mir-875,323,0.214430,0.050,3.933321,1.027043e-04
